In [1]:
#tf resnet packages + utilities
import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras import backend
import numpy as np
from tensorflow.keras.preprocessing import image

#sagemaker clients
import boto3
import json
import os
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')
role = sagemaker.get_execution_role()
region = boto_session.region_name
sagemaker_session = sagemaker.Session()
print(region)
print(role)

us-east-1
arn:aws:iam::474422712127:role/sagemaker-role-BYOC


In [2]:
export_dir = "00003"

In [3]:
tf.keras.backend.set_learning_phase(0)
model = tf.keras.applications.ResNet50()

if not os.path.exists(export_dir):
    os.makedirs(export_dir)
    print("Directory ", export_dir, " Created ")
else:
    print("Directory ", export_dir, " already exists")
# Save to SavedModel
model.save(export_dir, save_format="tf", include_optimizer=False)

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-11-14 19:01:45.546538: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-14 19:01:45.546581: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-36-57.ec2.internal): /proc/driver/nvidia/version does not exist
2022-11-14 19:01:45.547029: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-criti

Directory  00003  Created 


2022-11-14 19:01:52.053875: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 00003/assets


/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [4]:
!tar -cvpzf model.tar.gz ./00003

./00003/
./00003/variables/
./00003/variables/variables.data-00000-of-00001
./00003/variables/variables.index
./00003/keras_metadata.pb
./00003/saved_model.pb
./00003/assets/


In [5]:
!tar -cvpzf source.tar.gz inference.py

inference.py


In [6]:
#Bucket for model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

sagemaker-us-east-1-474422712127


In [7]:
s3 = boto3.client('s3')
for i in range(0,5):
    with open("model.tar.gz", "rb") as f:
        s3.upload_fileobj(f, default_bucket, "resnet-mme/resnet-{}.tar.gz".format(i))

In [8]:
!aws s3 ls s3://{default_bucket}/resnet-mme/

2022-11-14 19:05:31   95350219 resnet-0.tar.gz
2022-11-14 19:05:32   95350219 resnet-1.tar.gz
2022-11-14 19:05:33   95350219 resnet-2.tar.gz
2022-11-14 19:05:33   95350219 resnet-3.tar.gz
2022-11-14 19:05:34   95350219 resnet-4.tar.gz


In [9]:
model_url = 's3://{}/resnet-mme/'.format(default_bucket)
model_url

's3://sagemaker-us-east-1-474422712127/resnet-mme/'

In [10]:
!aws s3 cp source.tar.gz s3://{default_bucket}/resnet-script/source.tar.gz

upload: ./source.tar.gz to s3://sagemaker-us-east-1-474422712127/resnet-script/source.tar.gz


In [11]:
!aws s3 ls s3://{default_bucket}/resnet-script/

2022-11-14 19:05:49        137 source.tar.gz


In [12]:
source_dir = 's3://{}/resnet-script/'.format(default_bucket)
source_dir

's3://sagemaker-us-east-1-474422712127/resnet-script/'

In [13]:
# retrieve tf image
image_uri = sagemaker.image_uris.retrieve(
    framework="tensorflow",
    region="us-east-1",
    version="2.3.0",
    py_version="py3",
    instance_type="ml.m5.xlarge",
    image_scope="inference"
)
image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu'

In [14]:
from time import gmtime, strftime
model_name = 'mme-tf' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": image_uri,
            "Mode": "MultiModel",
            "ModelDataUrl": model_url,
            "Environment": {'SAGEMAKER_SUBMIT_DIRECTORY': source_dir,
                           'SAGEMAKER_PROGRAM': 'inference.py'} 
        }
    ],
    ExecutionRoleArn=role,
)
print("Model Arn: " + create_model_response["ModelArn"])

Model name: mme-tf2022-11-14-19-05-59
Model data Url: s3://sagemaker-us-east-1-474422712127/resnet-mme/
Model Arn: arn:aws:sagemaker:us-east-1:474422712127:model/mme-tf2022-11-14-19-05-59


In [15]:
#Step 2: EPC Creation
tf_epc_name = "mme-tf" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=tf_epc_name,
    ProductionVariants=[
        {
            "VariantName": "tfvariant",
            "ModelName": model_name,
            "InstanceType": "ml.c5.large",
            "InitialInstanceCount": 1
        },
    ],
)
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint-config/mme-tf2022-11-14-19-06-04


In [16]:
#Step 3: EP Creation
endpoint_name = "mme-resnet" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=tf_epc_name,
)
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-resnet2022-11-14-19-06-09


In [17]:
#Monitor creation
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)
print(describe_endpoint_response)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'mme-resnet2022-11-14-19-06-09', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:474422712127:endpoint/mme-resnet2022-11-14-19-06-09', 'EndpointConfigName': 'mme-tf2022-11-14-19-06-04', 'ProductionVariants': [{'VariantName': 'tfvariant', 'DeployedImages': [{'SpecifiedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.3.0-cpu', 'ResolvedImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference@sha256:91ebb7428846c5f7b515d5d9b8389a14c73d0c5d02657f4a6413592124333278', 'ResolutionTime': datetime.datetime(2022, 11, 14, 19, 6, 10, 661000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(2022, 11, 14, 19, 6, 10, 10000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2022, 11, 14, 19, 7, 48, 252000, tzinfo=tzlocal()), 'Resp

In [18]:
# resizing to 224x224 pixels (required by this model)
img = image.load_img("kitten.jpg", target_size=(224, 224))
# Convert image to a numpy array
x = image.img_to_array(img)
# Add a forth dimension since Keras expects a list of images
x = np.expand_dims(x, axis=0)

# resnet preprocessing
x = resnet50.preprocess_input(x)
print(type(x))

<class 'numpy.ndarray'>


In [19]:
payload = json.dumps(x.tolist())
print(type(payload))

<class 'str'>


In [20]:
endpoint_name

'mme-resnet2022-11-14-19-06-09'

In [24]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                   TargetModel='resnet-3.tar.gz',
                                   ContentType='application/json',
                                   Body=payload)
result = json.loads(response['Body'].read().decode())
#result